In [13]:
# Map Castilla y León cultural agenda geolocalized
import os
import folium  # Maps library
import requests # Library to make url requests
import time
import webbrowser
import bs4 # Web Scrapping
import random
import base64
%load_ext folium_magic

The folium_magic extension is already loaded. To reload it, use:
  %reload_ext folium_magic


In [14]:
# Web Scrapping

#cyl = "https://datosabiertos.jcyl.es/web/es/datos-abiertos-castilla-leon.html"
cyl = "https://datosabiertos.jcyl.es/web/jcyl/RISP/es/PlantillaBuscadorContenidos/1284162104333/Dataset/1284197734502/_?p0=false&p2=Dataset&tituloc=true&q=empleos&aceptar=Buscar"
def scrap(url):
    r  = requests.get(url)

    data = r.text

    soup = bs4.BeautifulSoup(data, "html.parser")
    
    return soup
soup = scrap(cyl)

# Locating website Ofertas de Empleo dataset
empleo = [x.get('href') for x in soup.findAll('a') if 'ofertas' in x.get('href')][0]
# Locating JSON file
soup = scrap(empleo)
cyl_json = [link.get('href') for link in soup.findAll('a') if "json" in link.get('href')][0]
print(cyl_json)


https://datosabiertos.jcyl.es/web/jcyl/risp/es/empleo/ofertas-empleo/1284354353012.json


In [23]:
"""
while True:
"""   
# Requesting the file
r = requests.get(cyl_json)
    
# Having the file as a JSON object
r = r.json()

#date = r['document']['date']
empleos, geolocs = r['document']['list'], []
for i in empleos:
    if 'LocalidadPadre' in i['element']['attribute'][9]:
        geolocs.append(i)
        #print(i['element']['attribute'][9]['LocalidadPadre'])
#print(geolocs[0]['element']['attribute'][9]['LocalidadPadre'].split('#')[:-1])

40.560574#-5.676764#88
41.401096#-4.314916#88
41.308163#-4.915477#88
41.52475#-5.394138#88
42.2972527905#-5.89591028588#88
41.652252#-4.724532#88
41.308163#-4.915477#88
41.76443#-2.463772#88
40.779846#-4.411779#88
42.4547895989#-6.05314829511#88
41.67027#-3.68877#88
41.76443#-2.463772#88
41.507324#-5.745313#88
40.9650951373#-5.66404069586#88
41.198772#-4.436803#88
42.3408235575#-3.70205869083#88
42.3408235575#-3.70205869083#88
42.866188#-5.323526#88
41.507324#-5.745313#88
40.46339#-5.328194#88
40.469257#-5.7207685#88
40.656685#-4.6812086#88
41.485928#-2.5331845#88
41.67027#-3.68877#88
40.46339#-5.328194#88
41.485928#-2.5331845#88
41.430702#-4.6644106#88
41.222366#-4.1737742#88
40.979843#-5.609179#88
42.3408235575#-3.70205869083#88
40.902122#-4.00729#88
40.8271496057#-5.51451226893#88
41.52542#-4.115328#88
41.570534#-3.1993084#88
41.364513#-2.1708138#88
40.2096926184#-5.09137409319#88
41.652252#-4.724532#88
40.8271496057#-5.51451226893#88
41.962326#-4.340458#88
42.3408235575#-3.70205869

In [33]:
# Creating a map
centro = geolocs[0]['element']['attribute'][9]['LocalidadPadre'].split('#')[:-1]
centro_posicion = [float(centro[0]),float(centro[1])]
folium_map = folium.Map(center=centro_posicion , zoom_start = 15000)
markers = []
# Creating a circle on the map
for i in geolocs:
    titulo = i['element']['attribute'][1]['text']
    fecha = i['element']['attribute'][3]['date']
    content = i['element']['attribute'][-1]['valor']
    latlong = i['element']['attribute'][9]['LocalidadPadre'].split('#')[:-1]
    if latlong!= []:
        posicion = [float(latlong[0]),float(latlong[1])]
    folium.Circle(
        radius=100,
        location= posicion,
        popup= content, 
        tooltip= titulo+'\n\n'+ fecha,
        color= '#'+str(random.randint(0,999999)),
        fill=True,
        ).add_to(folium_map)
    markers.append({'lat': posicion[0],'lng': posicion[1], "popup": titulo+'\n\n'+ fecha+'\n\n'+content,
                    "color" : '#'+str(random.randint(0,50000))})
    

In [34]:
%folium_map -M markers

In [18]:
# Saving the map to a HTML file to be read by a web browser
pagina = folium_map.save('cyl_empleos.html')

In [19]:
# Run html in webbrowser
webbrowser.open('file://'+ os.path.realpath('cyl_empleos.html'), new = -1)

False